In [ ]:
import numpy as np
from skimage.metrics import structural_similarity
import cv2
from imageio import imread
from random import random, choice, randint, sample
from PIL import Image
from PIL import ImageDraw
import os


class GeneticAlgorithm:

    def __init__(self, img_path, save_name="temp", top=5, max_group=200, features=100, epochs=1000):
        self.original_img, self.img_type, self.row, self.col = self.open_image(img_path)
        self.max_group = max_group
        self.top = top
        self.save_name = save_name
        self.groups = []
        self.features = features
        self.epochs = epochs
        self.group_dict = dict()

        if not os.path.exists(save_name):
            os.mkdir(save_name)

        self.initialize_groups()

    def open_image(self, img_path):
        img = imread(img_path)
        row, col = img.shape[0], img.shape[1]
        return img, img_path.split(".")[-1], row, col

    def initialize_groups(self):
        print("Initializing...")
        for _ in range(self.max_group):
            group = []
            for _ in range(self.features):
                polygon = [[choice(np.linspace(0, self.row, self.features)), choice(np.linspace(0, self.col, self.features))] for _ in range(3)]
                polygon.append("#" + ''.join(choice('0123456789ABCDEF') for _ in range(6)))

                group.append(polygon.copy())

            self.groups.append(group.copy())
        print("Initialization complete!")

    def to_image(self, group):
        array = np.ndarray((self.original_img.shape[0], self.original_img.shape[1], self.original_img.shape[2]), np.uint8)
        array[:, :, 0] = 255
        array[:, :, 1] = 255
        array[:, :, 2] = 255
        new_image = Image.fromarray(array)
        draw = ImageDraw.Draw(new_image)
        for polygon in group:
            draw.polygon((polygon[0][0], polygon[0][1], polygon[1][0], polygon[1][1], polygon[2][0], polygon[2][1]), polygon[3])

        return new_image

    def calculate_similarity(self, group) -> float:
        generated_image = self.to_image(group)
        ssim = structural_similarity(np.array(self.original_img), np.array(generated_image), multichannel=True)
        return ssim

    def save_generated_image(self, group, epoch):
        image = self.to_image(group)
        image.save(os.path.join(self.save_name, str(epoch) + "." + self.img_type))

    def crossover(self, father, mother):
        min_locate = min(len(father), len(mother))
        
        # Select a percentage of genes to perform crossover
        n = int(0.20 * min_locate)
        selected_indices = sample(range(min_locate), n)

        child1 = father.copy()
        child2 = mother.copy()

        for idx in selected_indices:
            # Get the gene from the father and the mother
            father_gene = father[idx]
            mother_gene = mother[idx]

            # Calculate the arithmetic average for each gene (only coordinates, not color)
            avg_gene = [[(father_gene[i][0] + mother_gene[i][0]) / 2,
                        (father_gene[i][1] + mother_gene[i][1]) / 2] for i in range(3)]

            # Assign the calculated average to the children
            for i in range(3):
                child1[idx][i] = avg_gene[i]
                child2[idx][i] = avg_gene[i]

            # Perform arithmetic crossover for the RGB color channels
            father_color = father_gene[-1]
            mother_color = mother_gene[-1]

            # Extract the RGB values from the hex color
            father_rgb = [int(father_color[i:i+2], 16) for i in (1, 3, 5)]
            mother_rgb = [int(mother_color[i:i+2], 16) for i in (1, 3, 5)]

            # Calculate the arithmetic average for each color channel (R, G, and B)
            avg_rgb = [int((father_rgb[i] + mother_rgb[i]) / 2) for i in range(3)]

            # Convert the averaged RGB values back to a hex color
            avg_color = '#' + ''.join(['{:02X}'.format(channel) for channel in avg_rgb])

            # Assign the calculated average color to the children
            child1[idx][-1] = avg_color
            child2[idx][-1] = avg_color

        return [child1, child2]

    def mutate(self, group):
        n = int(randint(1, 100) / 1000 * len(group))

        selected = sample(range(0, len(group)), n)

        for s in selected:
            polygon = [[choice(np.linspace(0, self.row, self.features)), choice(np.linspace(0, self.col, self.features))] for _ in range(3)]
            polygon.append("#" + ''.join(choice('0123456789ABCDEF') for _ in range(6)))
            group[s] = polygon

        return group

    def move_polygons(self, group):
        exchange = int(randint(1, 100) / 1000 * len(group))
        for _ in range(exchange):
            idx1 = randint(0, len(group) - 1)
            idx2 = randint(0, len(group) - 1)

            group[idx1], group[idx2] = group[idx2], group[idx1]

        return group

    def add_polygons(self, group):
        n = int(randint(1, 100) / 1000 * len(group))

        for _ in range(n):
            polygon = [
                [choice(np.linspace(0, self.row, self.features)),
                choice(np.linspace(0, self.col, self.features))]
                for _ in range(3)]
            polygon.append("#" + ''.join(choice('0123456789ABCDEF') for _ in range(6)))
            group.append(polygon)

        return group

    def remove_polygons(self, group):
        n = int(randint(1, 100) / 1000 * len(group))
        selected = sample(range(0, len(group)), n)

        new_group = []
        for idx in range(len(group)):
            if idx not in selected:
                new_group.append(group[idx])

        return new_group

    def generate_variations(self, group):
        mutated = self.mutate(group.copy())
        moved = self.move_polygons(mutated.copy())
        added = self.add_polygons(moved.copy())
        removed = self.remove_polygons(added.copy())
        return [mutated, moved, added, removed]

    def breed(self, parent1, parent2):
        offspring1, offspring2 = self.crossover(parent1.copy(), parent2.copy())

        offspring = []
        offspring.extend(self.generate_variations(parent1.copy()))
        offspring.extend(self.generate_variations(parent2.copy()))

        return offspring

    def eliminate_less_fit(self, groups):
        self.group_dict.clear()
        for idx in range(len(groups)):
            self.group_dict[idx] = self.calculate_similarity(groups[idx])

        self.group_dict = {key: value for key, value in
                          sorted(self.group_dict.items(), key=lambda item: item[1], reverse=True)}

        fittest_groups = []
        for key in list(self.group_dict.keys())[:self.max_group]:
            fittest_groups.append(self.groups[key].copy())

        groups = fittest_groups.copy()
        return groups, list(self.group_dict.values())[0]

    def run(self):
        # Run the genetic algorithm to recreate the input image
        self.eliminate_less_fit(self.groups)
        for epoch in range(self.epochs):
            # Breeding process
            breed_n = randint(self.max_group // 2, self.max_group)
            fitness_values = np.abs(np.array(list(self.group_dict.values())))
            probabilities = fitness_values / np.sum(fitness_values)
            for _ in range(breed_n):
                father_idx = np.random.choice(list(self.group_dict.keys()), p=probabilities.ravel())
                mother_idx = np.random.choice(list(self.group_dict.keys()), p=probabilities.ravel())
                if father_idx < self.max_group and mother_idx < self.max_group:
                    self.groups.extend(self.breed(self.groups[int(father_idx)].copy(), self.groups[int(mother_idx)].copy()))

            # Eliminate less fit groups
            self.groups, accuracy = self.eliminate_less_fit(self.groups.copy())
            print("Epochs :", epoch+1, " Accuracy:", accuracy)
            last_group = self.groups[0]

            if epoch % 100 == 0:
                self.save_generated_image(self.groups[0], epoch)

            if accuracy >= 0.95:
                break
        self.save_generated_image(self.groups[0], "End")


    # Define input parameters
path = '/content/test.png'
savename = 'test result'

# Create a Genetic_Algorithm instance and run the algorithm
population_size = 500
GA = GeneticAlgorithm(path, savename, 5, population_size, 50, 100000000)
GA.run()

<ipython-input-13-c7a0d70f06a6>:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(img_path)


Initializing...
Initialization complete!


<ipython-input-13-c7a0d70f06a6>:60: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim = structural_similarity(np.array(self.original_img), np.array(generated_image), multichannel=True)


Epochs : 1  Accuracy: 0.3922415459263463
Epochs : 2  Accuracy: 0.3981613348411983
Epochs : 3  Accuracy: 0.4171344314312049
Epochs : 4  Accuracy: 0.4171347644667768
Epochs : 5  Accuracy: 0.3919496181278517
Epochs : 6  Accuracy: 0.3946042929975
Epochs : 7  Accuracy: 0.3870188986996508
Epochs : 8  Accuracy: 0.3993860353102322
Epochs : 9  Accuracy: 0.39946381048180346
Epochs : 10  Accuracy: 0.3945964408540653
Epochs : 11  Accuracy: 0.42066099679342495
Epochs : 12  Accuracy: 0.4208180568314613
Epochs : 13  Accuracy: 0.4161000478786778
Epochs : 14  Accuracy: 0.4283428564391069
Epochs : 15  Accuracy: 0.44033310731624764
Epochs : 16  Accuracy: 0.44033310731624764
Epochs : 17  Accuracy: 0.3964300607111286
Epochs : 18  Accuracy: 0.3964300607111286
Epochs : 19  Accuracy: 0.4029919283266343
Epochs : 20  Accuracy: 0.4007149859167407
Epochs : 21  Accuracy: 0.43127252891471324
Epochs : 22  Accuracy: 0.43115295207464577
Epochs : 23  Accuracy: 0.42624570112646876
Epochs : 24  Accuracy: 0.42624429881678